## TO DO:
- [-] Take rough number of images to be downloaded as input. (Rough because few images will be corrupt and deleted later on.) - MAY NOT NEED THIS, SEE BELOW. We could also gather URLS and then select the number of URLs to scrape. But this would cap the number of images at 100, since the script scrapes 100 URLS by default, not counting the few corrupt images that we'd lose.
- [x] The make_image_dataset function should be able to take Lists as input. When taking list inputs, need to create folders within destination folder to segregate the images.

## Getting more data:
1. Tagged Anime Illustrations: <br>
https://www.kaggle.com/mylesoneill/tagged-anime-illustrations/home - Will need to clone this directly into GCP. 36 GB is too much for my pleb laptop.

2. Anime Faces : <br>
http://www.nurs.or.jp/~nagadomi/animeface-character-dataset/

3. Check out the links above they lead to tabular anime data as well.

### We can this the dataset gathered for Anime Identification. That is, if the above falls short.

## Lesson learnt:
## Save all FAST AI notebooks that you edit to your local storage.

In [1]:
#!pip install pyvirtualdisplay
#!pip install xvfbwrapper
#!sudo apt-get install xvfb

In [75]:
from imutils import paths
import argparse
import requests
import cv2
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import os
import glob
import time
from pyvirtualdisplay import Display
from threading import *
from multiprocessing import Pool, Process, Queue
from itertools import product
from functools import partial

In [4]:
#Anime_Faces_List = 

In [5]:
images_to_download_list = ["naruto", "cowboy bebop"]

In [127]:
def make_image_dataset(image_name, dest_folder = r"./anime_faces/", downloads_folder= r"/home/jupyter/Downloads/"):    
    #images_to_download = input("Which images do you want to download? ")
    #dest_folder = input("Where do you want the downloaded images? ")
    #downloads_folder = input("Enter the path to your downloads folder. ")
    
    os.mkdir(os.path.sep.join([dest_folder, image_name]))
    global dest_folder_new
    dest_folder_new = os.path.sep.join([dest_folder, image_name])
        
    options = webdriver.ChromeOptions()
    #options.add_argument('headless')
    #options.add_argument('window-size=1200x600')  
    display = Display(visible=0, size=(1200, 600))
    display.start()
        
    driver = webdriver.Chrome(executable_path = "/home/jupyter/chromedriver.exe", options=options)
    
    driver.get("https://www.google.com/search?tbm=isch&q=" + image_name)
    
    urls = driver.execute_script(r'''urls = Array.from(document.querySelectorAll('.rg_di .rg_meta')).map(el=>JSON.parse(el.textContent).ou);
    window.open('data:text/csv;charset=utf-8,' + escape(urls.join('\n')));''')
    time.sleep(4)
    
    driver.quit()
    display.stop()
    
    #list_of_files = glob.glob(r"C:\Users\kmoudgalya\Downloads\*") # * means all if need specific format then *.csv
    list_of_files = glob.glob(os.path.sep.join([downloads_folder, "/*"]))
    latest_file = max(list_of_files, key=lambda x: os.stat(x).st_mtime)
    
    rows = open(latest_file).read().strip().split("\n")
    
    
    #print(rows)
    
    global URL_downloader
    def URL_downloader(URL_rows):
        # loop the URLs
        total = 0
        for url in URL_rows:
            try:
                # try to download the image
                r = requests.get(url, timeout=10)

                # save the image to disk
                p = os.path.sep.join([dest_folder_new, "{}.jpg".format(str(total).zfill(8))])
                f = open(p, "wb")
                f.write(r.content)
                f.close()

                # update the counter
                print("[INFO] downloaded: {}".format(p))
                total += 1

            # handle if any exceptions are thrown during the download process
            except:
                print("[INFO] error downloading {}...skipping".format(p))

#    URL_downloader(rows)
    x = Pool(10)
    records = x.map(URL_downloader, [rows])
    x.terminate()
    x.join()    
    
    
    global Image_Checker
    def Image_Checker(folder):
        # loop over the image paths we just downloaded
        for imagePath in paths.list_images(folder):
            # initialize if the image should be deleted or not
            delete = False

            # try to load the image
            try:
                image = cv2.imread(imagePath)

                # if the image is `None` then we could not properly load it
                # from disk, so delete it
                if image is None:
                    delete = True

            # if OpenCV cannot load the image then the image is likely
            # corrupt so we should delete it
            except:
                print("Except")
                delete = True

            # check to see if the image should be deleted
            if delete:
                print("[INFO] deleting {}".format(imagePath))
                os.remove(imagePath)

#    Image_Checker(os.path.sep.join([dest_folder, image_name]))
    y = Pool(10)
    records_2 = y.map(Image_Checker, [dest_folder_new])
    y.terminate()
    y.join()
    
    
    print("Images downloaded. Let the Machine Learning begin.")

In [128]:
!rm -r anime_faces
!mkdir anime_faces

In [129]:
start_time = time.time()
make_image_dataset("naruto")
print('Entire job took:',time.time() - start_time)

[INFO] downloaded: ./anime_faces//naruto/00000000.jpg
[INFO] downloaded: ./anime_faces//naruto/00000001.jpg
[INFO] downloaded: ./anime_faces//naruto/00000002.jpg
[INFO] downloaded: ./anime_faces//naruto/00000003.jpg
[INFO] downloaded: ./anime_faces//naruto/00000004.jpg
[INFO] downloaded: ./anime_faces//naruto/00000005.jpg
[INFO] downloaded: ./anime_faces//naruto/00000006.jpg
[INFO] downloaded: ./anime_faces//naruto/00000007.jpg
[INFO] downloaded: ./anime_faces//naruto/00000008.jpg
[INFO] downloaded: ./anime_faces//naruto/00000009.jpg
[INFO] downloaded: ./anime_faces//naruto/00000010.jpg
[INFO] downloaded: ./anime_faces//naruto/00000011.jpg
[INFO] downloaded: ./anime_faces//naruto/00000012.jpg
[INFO] downloaded: ./anime_faces//naruto/00000013.jpg
[INFO] downloaded: ./anime_faces//naruto/00000014.jpg
[INFO] downloaded: ./anime_faces//naruto/00000015.jpg
[INFO] downloaded: ./anime_faces//naruto/00000016.jpg
[INFO] downloaded: ./anime_faces//naruto/00000017.jpg
[INFO] downloaded: ./anime_f

### When Pool = 5, 26s.
### When Pool = 10, 26s.
### When Pool = 20, 26s.
### For Naruto images.

### I guess at least it's consistently 26s seconds now, without multiprocessing it would vary from 28-32s.

### Time taken by function:
#### Naruto images download      : 32.81 s <br>
#### Cowboy Bepop images download: 40.70 s <br>
#### Total time                  : 73.51 s <br>

In [134]:
!rm -r anime_faces
!mkdir anime_faces

### We get some sort of race condition with the multiproccess + multhread attempt here.

In [133]:
start_time = time.time()
threadlist = []
for j in images_to_download_list:
    print(j)
    t = Thread(target=make_image_dataset, args=(j,))
    
    t.start()
    threadlist.append(t)
    
for b in threadlist:
   b.join()
    
print('Entire job took:',time.time() - start_time)

naruto
cowboy bebop
[INFO] downloaded: ./anime_faces//cowboy bebop/00000000.jpg
[INFO] downloaded: ./anime_faces//cowboy bebop/00000001.jpg
[INFO] downloaded: ./anime_faces//cowboy bebop/00000002.jpg
[INFO] downloaded: ./anime_faces//cowboy bebop/00000003.jpg
[INFO] downloaded: ./anime_faces//cowboy bebop/00000004.jpg
[INFO] downloaded: ./anime_faces//cowboy bebop/00000005.jpg
[INFO] downloaded: ./anime_faces//cowboy bebop/00000006.jpg
[INFO] downloaded: ./anime_faces//cowboy bebop/00000007.jpg
[INFO] downloaded: ./anime_faces//cowboy bebop/00000008.jpg
[INFO] downloaded: ./anime_faces//cowboy bebop/00000009.jpg
[INFO] downloaded: ./anime_faces//cowboy bebop/00000010.jpg
[INFO] downloaded: ./anime_faces//cowboy bebop/00000000.jpg
[INFO] downloaded: ./anime_faces//cowboy bebop/00000001.jpg
[INFO] downloaded: ./anime_faces//cowboy bebop/00000002.jpg
[INFO] downloaded: ./anime_faces//cowboy bebop/00000011.jpg
[INFO] downloaded: ./anime_faces//cowboy bebop/00000012.jpg
[INFO] downloaded: .

Exception in thread Thread-120:
multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-127-8a1c045d064e>", line 91, in Image_Checker
    os.remove(imagePath)
FileNotFoundError: [Errno 2] No such file or directory: './anime_faces//cowboy bebop/00000089.jpg'
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-127-8a1c045d064e>", line 95, in make_image_dataset
    records_2 = y.map(Image_Checker, [dest_folder_new])
  File "/o

### 24.05s (multi-threaded) vs 73.51s for the script to download images. I'd say this is a no brainer. 3x Improvement ftw! This is with 2 separate image targets (naruto and cowboy bepop).

### The script simply pauses if one puts down the laptop lid and resumes when the user logs back in to the laptop. Phew.

### 100x100 images (10000) take around 6-7 hours to be downloaded. Without Multithreading ofc.